# Homework 1

## 1.1
Using introspection alone to understand human cognitive processes would have limited usefulness as human cognitive 
processes are generally not conscious. For instance, trying to model the way a human throws a ball
purely through introspection would be almost impossible as so much of our physical action happens unconsciously.
Furthermore, It is common for people to not understand why they made a decision they made. Humans are often not
privy to their own cognitive processes making it difficult to use introspection to
gain much insight into our processes. When trying to make an AI that follows
the human thinking approach, introspection would have some usefulness but should probably be combined with other methods
of understanding the human mind.

## 1.2

Each state in my formulation is a list of cities, a path the salesman
would go along. Each path, even after actions done to it, should always
have the same starting and end point. The action the formulation can 
take is to swap two neighboring in the path cities in the order (as they are all interconnected).
In order to calculate the value, I am using the search.py graph class's get()
function which returns the distance between two nodes (which is also what I am using to place the cities randomly). 
I than multiply this by -1 to have the algorithms head towards the lowest value.



In [3]:
from search import Problem, hill_climbing, simulated_annealing, exp_schedule, RandomGraph
import random

class TSP(Problem):
    """An implementation of TSP for local search.
        State representation:
            [city0, city1, city2]
        Move representation:
            [index0, index1]: Swap the city in the given path index with the city in the other index.
    """
    def __init__(self, initialPath, cityGraph):
        self.initial = initialPath
        self.cityGraph = cityGraph

    def actions(self, state):
        """Actions: swap two cities next to each other in the path (as long as the first and last city are not altered)
        """
        actions = []
        for i in range(1, len(state)-2):
            newPath = state[:]
            newPath[i], newPath[i+1] = newPath[i+1], newPath[i]
            actions.append(newPath)
        return actions

    def result(self, state, move):
        """Makes the given move."""
        return move

    def value(self, state):
        """This method computes a value of given state using distance traveled
        """
        value = 0;
        for i in range(len(state)-1):
            value = value + cityGraph.get(state[i],state[i+1])
        # multiply by one to pursue the lowest value
        return -1 * value

In [4]:
# Set the number of cities
cityAmount = 20
print('Cities:\t\t' + str(cityAmount))

# Create unique city ids and add them to city list
cityList = []
for i in range(cityAmount):
    cityList.append(str(i))

# Create a random graph using search RandomGraph()
cityGraph = RandomGraph(cityList, cityAmount-1, 10000, 10000)

# Initialize initial path randomly
path = random.sample(cityGraph.nodes(), len(cityGraph.nodes()))

# Add first city as the last city to make it the destination
path.append(path[0])
print('Starting path:\t' + str(path))

p = TSP(path,cityGraph)

# Solve the problem using hill climbing.
hill_solution = hill_climbing(p)
print('\nHill-climbing:')
print('\tSolution:\t' + str(hill_solution))
print('\tValue:\t\t' + str(p.value(hill_solution)))

# Solve the problem using simulated annealing.
annealing_solution = \
    simulated_annealing(p, exp_schedule(k=20, lam=0.005, limit=10000))
print('\nSimulated annealing:')
print('\tSolution:\t' + str(annealing_solution))
print('\tValue:\t\t' + str(p.value(annealing_solution)))

Cities:		20
Starting path:	['14', '5', '4', '0', '9', '7', '18', '2', '11', '8', '10', '13', '15', '1', '12', '6', '3', '16', '17', '19', '14']

Hill-climbing:
	Solution:	['14', '5', '4', '0', '9', '7', '18', '11', '2', '1', '8', '10', '13', '15', '12', '6', '3', '16', '17', '19', '14']
	Value:		-101010

Simulated annealing:
	Solution:	['14', '5', '4', '0', '9', '7', '18', '11', '10', '2', '8', '13', '15', '12', '1', '6', '3', '16', '17', '19', '14']
	Value:		-112266


My sample city domain is randomly generated using the pre-made RandomGraph function provided in search (I also make a random path
with the start and end being the same city). After trying hill-climbing and simulated annealing on my city,
both searches result in an answer very quickly at small city numbers but generally hill climbing
results in a better answer (at least with a relatively small number of cities I have tested), 
possibly because this problem (at least in the interconnected manner we formulated it) does not have very many local maximums.

## 1.3
My formulation of the course_scheduling problem uses one variable, course, which has two values: its room and its 
time slot. Course assignments are also required to check in the course constraint. I chose one variable, the course, 
as this ensures that each course will only occur once. I thought about adding a professor value to the domain but
I realized that this was unnecessary as problems with professors only occur if they are teaching two classes at the same time which can be easily checked in the 
course_constraint function. 


In [5]:
import itertools
from csp import CSP, backtracking_search


def course_scheduling(courses, course_assignments, rooms, time_slots):
    """Return an instance our cs course scheduling problem."""
    variables = courses
    domains = {}
    domains.fromkeys(variables)
    for course in range(len(courses)):
        # give each course a list within which to put all possible values it can have
        domains[courses[course]] = list(itertools.product(rooms, time_slots))

    # every course is every other courses' neighbor
    neighbors = {}
    for course in courses:
        neighbors[course] = [other_course for other_course in courses if other_course != course]

    def course_constraint(A, a, B, b):
        if a[1] == b[1]:
            # The time is the same
            if a[0] == b[0]:
                # Two classes take place at the same place at the same time
                return False
            if course_assignments[A] == course_assignments[B]:
                # You have a professor at two places at the same time
                return False
        return True

    return CSP(variables, domains, neighbors, course_constraint)

Here I demonstrate my problem works with backtracking search and results in a good answer for the provided schedule requirements.

In [6]:
courses = 'cs108 cs112 cs212 cs214 cs232 cs344'.split()
course_assignments = {'cs108': 'Dschuurman', 'cs112': 'Adams', 'cs212': 'Dschuurman', 'cs214': 'Adams', 'cs232': 'Norman', 'cs344': 'VanderLinden'}
rooms = 'nh253 sb382'.split()
time_slots = 'mwf900 mwf1130 tth900 tth1030'.split()
p = course_scheduling(courses, course_assignments, rooms, time_slots)
# backtracking search
back_result = backtracking_search(p)
if p.goal_test(p.infer_assignment()):
    print("Backtracking Solution:\n")
    print(p.curr_domains)
else:
    print("Backtracking failed...")
    print(p.curr_domains)

Backtracking Solution:

{'cs108': [('nh253', 'tth900')], 'cs112': [('sb382', 'mwf900')], 'cs212': [('nh253', 'mwf1130')], 'cs214': [('sb382', 'mwf1130')], 'cs232': [('nh253', 'tth1030')], 'cs344': [('nh253', 'mwf900')]}
